## using normal clustering methods to try and find tag hierarchy structure


### [scipy.linkage](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html)

### [scipy.dendrogram](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html)

## results: mostly bad. Top-level clusters are "java", "javascript", "php" etc, not "databases" and "security" and general "tags" like that. 


In [3]:
import csv
import os
import pickle
import re
import sys
import numpy as np

from joblib import Parallel, delayed

from difflib import SequenceMatcher,get_close_matches

from scipy import spatial

from tqdm import *

module_path = os.path.abspath(os.path.join('../helpers/'))
if module_path not in sys.path:
    sys.path.append(module_path)

# my stuff in the helpers/ directory
import embeddings_helper, files_helper, texts_helper, metrics_helper, tags_helper, cobrinha_helper

from cobrinha_helper import get_metrics_for_tag_pair

In [4]:
PICKLE_ROOT = "/media/felipe/SSD_VOLUME//auto-tagger/data/tag-hierarchy/"

tag_vocabulary = pickle.load(open(PICKLE_ROOT+"tag_vocabulary.p","rb"))
tag_vectors_index = pickle.load(open(PICKLE_ROOT+"tag_vectors_index.p","rb"))

In [5]:
tag_vectors_index['sql']

array([ 13.31756595,   1.2150911 ,   0.30977384, ...,   0.        ,
         0.        ,   0.        ])

sort the tag vectors by name

In [6]:
tag_vectors_as_list = list()

for key, value in tag_vectors_index.items():
    tag_vectors_as_list.append((key,value))
   
ordered_tag_vectors = sorted(tag_vectors_as_list, key= lambda tpl: tpl[0])    

### without any external information or connectivity matrix

In [7]:
from scipy.cluster.hierarchy import dendrogram, linkage
import matplotlib.pyplot as plt

In [8]:
points = list()
labels = list()

for key,value in ordered_tag_vectors:
    points.append(value)
    labels.append(key)
    
data = np.vstack(points)

data.shape

(618, 4692)

In [9]:
labels[:10]

['.htaccess',
 '.net',
 '.net-4.0',
 '3d',
 'access-vba',
 'actionscript-3',
 'active-directory',
 'activerecord',
 'ajax',
 'algorithm']

In [23]:
W=50
H=20
P=5
TRUNCATE_MODE='level'
LINKAGE='single'

plt.figure(figsize=(W,H))

Z = linkage(data,LINKAGE)

dn = dendrogram(Z,labels=labels,truncate_mode=TRUNCATE_MODE,p=P,leaf_font_size=40)

fig = plt.gcf()

ax = plt.gca()

ax.tick_params(axis='x', which='major', labelsize=40)
ax.tick_params(axis='y', which='major', labelsize=40)

fig.savefig("dendrograms/{}-{}-{}.svg".format(LINKAGE,TRUNCATE_MODE,P), format='svg')


analysis only

In [11]:
Z[7]

array([ 492.        ,  493.        ,   11.66433737,    2.        ])

In [12]:
dn2 = dendrogram(Z,labels=labels)

In [13]:
dn2["leaves"]

[13,
 406,
 286,
 424,
 296,
 75,
 120,
 247,
 272,
 27,
 362,
 72,
 488,
 428,
 149,
 213,
 450,
 39,
 41,
 449,
 440,
 302,
 371,
 609,
 279,
 380,
 489,
 8,
 509,
 599,
 163,
 563,
 600,
 337,
 354,
 456,
 360,
 176,
 184,
 319,
 404,
 483,
 504,
 608,
 388,
 129,
 591,
 416,
 58,
 178,
 565,
 340,
 332,
 427,
 536,
 257,
 511,
 171,
 61,
 9,
 141,
 248,
 153,
 391,
 202,
 26,
 167,
 317,
 33,
 512,
 321,
 433,
 241,
 546,
 579,
 238,
 415,
 114,
 244,
 0,
 401,
 310,
 123,
 495,
 122,
 97,
 34,
 508,
 346,
 62,
 571,
 597,
 164,
 222,
 479,
 400,
 126,
 185,
 291,
 551,
 613,
 168,
 383,
 425,
 426,
 394,
 583,
 218,
 227,
 304,
 182,
 472,
 403,
 78,
 369,
 23,
 475,
 530,
 18,
 561,
 12,
 125,
 166,
 59,
 35,
 444,
 205,
 376,
 220,
 573,
 575,
 187,
 5,
 194,
 20,
 518,
 507,
 323,
 243,
 237,
 481,
 226,
 136,
 138,
 476,
 356,
 370,
 480,
 254,
 76,
 113,
 606,
 223,
 193,
 451,
 285,
 540,
 52,
 249,
 82,
 569,
 83,
 539,
 181,
 299,
 466,
 148,
 365,
 268,
 552,
 473,
 338,
